In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('CE.BC.E.up-to-o3.csv')
data.loc[:, 'Energy'] *= 1e-6 # Convert energy to Joules
benchmarks = data["Benchmark"].unique()
average_data = data.groupby(['Benchmark','Flags'], as_index=False).agg({'Energy':'mean', 'Time':'mean'})
average_data.head()

In [ ]:
def flag_diff(index, data):
    current = data.iloc[index]["Flags"].split(' ')
    previous = data.iloc[index - 1]["Flags"].split(' ')
    
    return set(current) - set(previous)

def initial_baseline(variable, data):
    return data.loc[(~data["Flags"].str.contains('no')) & (data["Flags"] != '-O3')].iloc[0][variable]

In [ ]:
def flags_with_no_effect(benchmark, variable, data, tolerance):
    benchmark_data = data.loc[data["Benchmark"] == benchmark]
    benchmark_data.head()

    flags = set()

    baseline = initial_baseline(variable, benchmark_data)
    for i, measurement in enumerate(benchmark_data[variable].values):
        if abs((measurement - baseline) / baseline) <= tolerance :
            flags = flags.union(flag_diff(i, benchmark_data))

    return flags

In [ ]:
flags = []

for benchmark in benchmarks:
    flags.append(flags_with_no_effect(benchmark, 'Energy', average_data, tolerance=0.0001))

print(set.intersection(*flags))